# Chord

> Stack of notes

In [ ]:
#|default_exp chord

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import numpy as np
import pandas as pd
from fastcore.all import *
from IPython.display import Audio
from mingus.core import chords as mingus_chords

from musy import Note, Interval

The `Chord` class is a collection of notes played together. The name of the chord is automatically inferred from the notes.

In [ ]:
#|export
class Chord(BasicRepr):
    def __init__(self, notes: List[Note]):
        self.notes = [Note(n) if isinstance(n, str) else n for n in notes]
        self.s_notes = [str(n) for n in self.notes]
        self.short_s_notes = [f"{n.note}{n.oct}" for n in self.notes]
        self.first = self.notes[0]
        self.s_first = str(self.first)

    def names(self) -> list[str]: return mingus_chords.determine(self.s_notes)
    def name(self): return self.names()[0] if self.names() else "No chord found."

    @classmethod
    def from_short(cls, c: str): return cls(mingus_chords.from_shorthand(c)) 

    def __repr__(self): return f"Chord: '{self.name()}'. Notes: {self.short_s_notes}"
    def __add__(self, other): return Chord([n + other for n in self.notes])
    def __sub__(self, other): return Chord([n - other for n in self.notes])
    def __mod__(self, other): return Chord([n % other for n in self.notes])
    def __len__(self): return len(self.notes)
    def __floordiv__(self, other): return Chord([n // other for n in self.notes])
    def __iter__(self) -> list[str]: return iter(self.notes)
    
    def _compare_notes(self, other, op): return all(op(n1, n2) for n1, n2 in zip(self.notes, other.notes))
    def __eq__(self, other): return self.first == other.first and self._compare_notes(other, lambda x, y: x == y)
    def __ne__(self, other): return not self == other
    def __lt__(self, other): return self.first < other.first or (self.first == other.first and self._compare_notes(other, lambda x, y: x < y))
    def __le__(self, other): return self.first < other.first or (self.first == other.first and self._compare_notes(other, lambda x, y: x <= y))
    def __gt__(self, other): return self.first > other.first or (self.first == other.first and self._compare_notes(other, lambda x, y: x > y))
    def __ge__(self, other): return self.first > other.first or (self.first == other.first and self._compare_notes(other, lambda x, y: x >= y))

## Initialization

In [ ]:
c_major = Chord(["C", "E", "G"])
c_major

Chord: 'C major triad'. Notes: ['C4', 'E4', 'G4']

`Chord` objects can be created from a string.

In [ ]:
cmaj7 = Chord.from_short("Cmaj7")
cmaj7

Chord: 'C major seventh'. Notes: ['C4', 'E4', 'G4', 'B4']

## Comparison

`Chord` objects can be compared to each other using familiar Python operators. The length of the chords and the underlying notes are compared. For example, A C major chord is technically lower than a C major 7th chord. The 1st 3 notes are the same, but Cmaj7 has an additional 4th note.

In [ ]:
assert c_major <= cmaj7

Length is only a tie breaker in this example. For example, an E major chord is higher than a D major 7 chord, because its root note is higher.

In [ ]:
assert Chord.from_short("E") >= Chord.from_short("Dmaj7")

## Transposition

### Semitones

`Chord` objects can be transposed in the same way as `Note` objects.

In [ ]:
# Cmaj2 + 2 semitones == Dmaj7
cmaj7 + 2

Chord: 'D major seventh'. Notes: ['D4', 'F#4', 'A4', 'C#5']

In [ ]:
# Dmaj7 > Cmaj7
cmaj7 + 2 > cmaj7

True

In [ ]:
# Cmaj7 - 1 == Bmaj7
cmaj7 - 1

Chord: 'B major seventh'. Notes: ['B4', 'D#4', 'F#4', 'A#3']

### Whole Notes

As with `Note` objects, there are shortcuts for transposing whole notes up and down by using the `%` and `//` operators, respectively.

In [ ]:
assert cmaj7 + 2 == cmaj7 % 1
# Transpose up 1 whole note
cmaj7 % 1

Chord: 'D major seventh'. Notes: ['D4', 'F#4', 'A4', 'C#5']

In [ ]:
# Transpose down 5 whole notes (Same Dmaj7 chord but 1 octave lower)
cmaj7 // 5

Chord: 'D major seventh'. Notes: ['D4', 'F#3', 'A3', 'C#3']

### Note/Note and Chord/Note Multiplication

`Note` objects can be multiplied with other `Note` objects to form a `Chord`. Multiplying `Chord` objects with `Note` objects will add the note to the chord.

In [ ]:
#|export
@patch
def __mul__(self:Note, other: Note):
    """ Multiply two notes to form a chord. """
    return Chord([self, other])

@patch
def __mul__(self:Chord, other):
    if isinstance(other, Note):
        return Chord(self.notes + [other])
    else:
        raise ValueError("Chord objects can only be multiplied with Note or other Chord objects")

In [ ]:
# Eb/C slash chord
eb_over_c = Note("Eb", oct=3) * Note("C", oct=4) * Note("E", oct=4) * Note("G", oct=4)
eb_over_c

Chord: 'No chord found.'. Notes: ['Eb3', 'C4', 'E4', 'G4']

## Inversion

`Chord` objects can be inverted with `invert`.

In [ ]:
#|export
@patch
def invert(self:Chord, n: int = 1):
    assert n > 0 and n < len(self.s_notes), f"Invalid inversion '{n}' for chord with '{len(self.s_notes)}' notes."
    return Chord(self.notes[n:] + [Note(str(note), oct=note.oct + 1) for note in self.notes[:n]])

In [ ]:
cmaj7.invert(2)

Chord: 'C major seventh, second inversion'. Notes: ['G4', 'B4', 'C5', 'E5']

## Intervals

`Interval` objects can be obtained for a `Chord`.

**Relative intervals** means we start from the root note and calculate all the intervals from it.

**Absolute intervals** means we calculate the intervals between the notes.



In [ ]:
#|export
@patch
def rel_intervals(self:Chord):
    return [Interval(self.notes[0], n) for n in self.notes[1:]]

@patch
def abs_intervals(self:Chord):
    return [Interval(n1, n2) for n1, n2 in zip(self.notes, self.notes[1:])]

In [ ]:
cmaj7_rel_intvals = cmaj7.rel_intervals()
assert len(cmaj7_rel_intvals) == 3
assert cmaj7_rel_intvals[-1].short == "7"
cmaj7_rel_intvals

[major third (3), perfect fifth (5), major seventh (7)]

In [ ]:
cmaj7_abs_intvals = cmaj7.abs_intervals()
assert len(cmaj7_abs_intvals) == 3
assert cmaj7_abs_intvals[-1].short == "3"
cmaj7_abs_intvals

[major third (3), minor third (b3), major third (3)]

## Audio

`Chord` objects can be played, just like `Note` objects.

In [ ]:
#|export
@patch
def get_audio_array(self:Chord, length=1):
    return np.sum([n.get_audio_array(length) for n in self.notes], axis=0)

@patch
def play(self:Chord, length=1): 
    return Audio(self.get_audio_array(length), rate=44100)

In [ ]:
cmaj7.play()

In [ ]:
cmaj7.invert(1).play()

In [ ]:
eb_over_c.play()

In [ ]:
Chord.from_short("Dbdim7").play()

## Chord Table

We can display all the relevant information about a chord in a Pandas DataFrame table.

In [ ]:
#|export
@patch
def to_frame(self:Chord):
    rel_intervals = self.rel_intervals()
    rel_short_intvals = [i.short for i in rel_intervals]
    rel_long_intvals = [i.long for i in rel_intervals]
    abs_intervals = self.abs_intervals()
    abs_short_intvals = [i.short for i in abs_intervals]
    abs_long_intvals = [i.long for i in abs_intervals]
    
    d = {
        "Notes": self.notes,
        "Relative Degree": [1] + rel_short_intvals,
        "Relative Interval": ["unison"] + rel_long_intvals,
        "Absolute Interval": ["unison"] + abs_long_intvals,
        "Absolute Degree": [1] + abs_short_intvals,
    }
    return pd.DataFrame(d)

In [ ]:
cmaj7.to_frame()

,Notes,Relative Degree,Relative Interval,Absolute Interval,Absolute Degree
0,C,1,unison,unison,1
1,E,3,major third,major third,3
2,G,5,perfect fifth,minor third,b3
3,B,7,major seventh,major third,3


Visualizing the chord as a table gives us a nice overview for analysis. 

For example, in the table for the `Cdim6maj7` chord below we can readily see that it constitutes of:
- A diminished triad (minor third (`b3`), and tritone (`b5`)), 
- a major sixth (`6`) and
- a major seventh (`maj7`).

In [ ]:
Cdim6maj7 = Chord([Note("C"), Note("D#"), Note("F#"), Note("A"), Note("B")])
Cdim6maj7.to_frame()

,Notes,Relative Degree,Relative Interval,Absolute Interval,Absolute Degree
0,C,1,unison,unison,1
1,D#,b3,minor third,minor third,b3
2,F#,b5,tritone,minor third,b3
3,A,6,major sixth,minor third,b3
4,B,7,major seventh,major second,2


# PolyChord

A `PolyChord` is a combination of notes. Much of the functionality is inherited from the `Chord` object.

In [ ]:
#|export
class PolyChord(Chord):
    def __init__(self, chords: list[Chord]):
        self.chords = chords
        super().__init__([note for chord in chords for note in chord.notes])
    def __repr__(self): return f"PolyChord: '{'|'.join([c.name() for c in self.chords])}'. Notes: {self.short_s_notes}"

## Initialization

In [ ]:
asus4 = Chord(Note(n, oct=5) for n in ["A", "D", "E"])
poly_chord = PolyChord([cmaj7, asus4])
poly_chord

PolyChord: 'C major seventh|A suspended fourth triad'. Notes: ['C4', 'E4', 'G4', 'B4', 'A5', 'D5', 'E5']

## Inversion

Like `Chord` objects, `PolyChord` objects can be inverted.

In [ ]:
#|export
@patch
def invert(self:PolyChord, n: int = 1):
    return PolyChord([c.invert(n) for c in self.chords])

In [ ]:
poly_chord.invert(1)

PolyChord: 'C major seventh, first inversion|D suspended second triad'. Notes: ['E4', 'G4', 'B4', 'C5', 'D5', 'E5', 'A6']

In [ ]:
poly_chord.rel_intervals()

[major third (3),
 perfect fifth (5),
 major seventh (7),
 major sixth (6),
 major second (2),
 major third (3)]

In [ ]:
poly_chord.play()

## Table

For the table display of a `PolyChord` we analyze the underlying chords separately.

In [ ]:
#|export
@patch
def to_frame(self:PolyChord) -> list[pd.DataFrame]:
    return [c.to_frame() for c in self.chords]

In [ ]:
[display(t) for t in poly_chord.to_frame()];

,Notes,Relative Degree,Relative Interval,Absolute Interval,Absolute Degree
0,C,1,unison,unison,1
1,E,3,major third,major third,3
2,G,5,perfect fifth,minor third,b3
3,B,7,major seventh,major third,3


,Notes,Relative Degree,Relative Interval,Absolute Interval,Absolute Degree
0,A,1,unison,unison,1
1,D,4,perfect fourth,perfect fourth,4
2,E,5,perfect fifth,major second,2


TODO: Check which scales/modes the chord belongs to.

FIX: Identify all diatonic chords in a scale.

TODO: Check if chord is diatonic within a scale.

-------------------------

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()